In [1]:
import pandas as pd
import numpy as np
import os

# Drawing
import cartopy
import matplotlib
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io import shapereader
from matplotlib.cm import get_cmap
import matplotlib.cm as cm
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [2]:
# data_df = pd.read_pickle("pickles/2014_15.pkl")
# council_dict = (data_df[['Council', 'count_people']].set_index('Council').T.to_dict('records'))[0]
# council_dict

## Map

In [3]:
def make_map(data_df, title, scale_max, filepath):
    fig = plt.figure(figsize=(6,8), dpi=100)
    projectionPARAM = ccrs.TransverseMercator(central_longitude=-2.0, central_latitude=49.0, false_easting=400000.0, false_northing=-100000.0, scale_factor=0.9996012717, approx=False)
    ax = fig.add_subplot(1, 1, 1, projection=projectionPARAM)
    ax.set_extent([-10.75, 2, 49.75, 61])

    council_dict = (data_df[['Council', 'count_people']].set_index('Council').T.to_dict('records'))[0]

    # Downloaded from: https://geoportal.statistics.gov.uk/datasets/ons::local-authority-districts-may-2022-uk-bfe-v3-1/about
    area_file = os.path.join(os.getcwd(), 'shapefiles', 'Local_Authority_Districts_May_2022_UK_BFE_V3_2022', 'LAD_MAY_2022_UK_BFE_V3.shp')
    council_divisions = shapereader.Reader(area_file)

    norm = colors.Normalize(vmin=0., vmax=scale_max)
    cmap = matplotlib.colormaps['plasma'] # PuBu
    
    # Counter for council_entries used
    council_entry_used = 0

    for record in council_divisions.records():
        code = record.attributes['LAD22NM']
        council_entry = council_dict.get(code, -1)
        if council_entry == -1:
            council_color = "Silver"
        else:
            council_color = cmap(council_entry/max(council_dict.values()))
            council_entry_used += 1
        
        ax.add_geometries(
                [record.geometry],
                #facecolor=numpy.random.rand(3,),
                facecolor=council_color,
                linewidth=0,
                crs=projectionPARAM,
        )

    divider = make_axes_locatable(ax)
    ax_cb = divider.new_horizontal(size="5%", pad=0.1, axes_class=plt.Axes)

    fig.add_axes(ax_cb)

    sm = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
    cb = plt.colorbar(sm, cax=ax_cb)
    cb.set_label("Number of employees in reciept of over £100,000 Total Renumeration")

    fig.suptitle(title, fontsize=16)
    
    # Print statistics
    print("We used " + str(round((council_entry_used/len(council_dict))*100, 2)) + "% of the council_entries")
    print("This was " + str(len(council_dict) - council_entry_used) + " councils that we couldn't match to the Shapefile")

    plt.tight_layout()
    plt.savefig(filepath)
    plt.close()

In [4]:
# data_df = pd.read_pickle("pickles/2021_22.pkl")
# make_map(data_df, "Town Hall Rich List 2023", max(data_df["count_people"]), "results/thrl_2023.png")

## Animated Map

In [5]:
import imageio

In [7]:
# TODO: Make version that fades between years!
n_frames = 10
start_year = 2000

# Create MAX for scale
current_max = -1
for i in range(11, 22):
    pickle_path = str(start_year+i) + "_" + str(i+1)
    data_df = pd.read_pickle("pickles/" + str(pickle_path) + ".pkl")
    
    this_max = max(data_df["count_people"])
    
    # Replace if bigger
    if current_max < this_max:
        current_max = this_max

print("Max determined on scale is: " + str(current_max))
# Create charts
print('Creating charts\n')
filenames = []
for i in range(11, 22):
    pickle_path = str(start_year+i) + "_" + str(i+1)
    
    data_df = pd.read_pickle("pickles/" + str(pickle_path) + ".pkl")
    
    filename = f'temporary_images/frame_{pickle_path}.png'
    make_map(data_df, pickle_path.replace("_", " - "), current_max, filename)
    
    for j in np.arange(0, n_frames + 1):
        filenames.append(filename)
        # last frame of each viz stays longer
        if (j == n_frames):
            for i in range(5):
                filenames.append(filename)
                
    print("Completed: " + str(pickle_path))

print('Charts saved\n')
# Build GIF
print('Creating gif\n')
with imageio.get_writer('results/static_version.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
print('Gif saved\n')
print('Removing Images\n')
# Remove files
for filename in set(filenames):
    os.remove(filename)
print('DONE')

Max determined on scale is: 68
Creating charts

We used 0.8379052369077307% of the council_entries
This was 65 councils that we couldn't match to the Shapefile
Completed: 2011_12
We used 0.8459595959595959% of the council_entries
This was 61 councils that we couldn't match to the Shapefile
Completed: 2012_13
We used 0.8229426433915212% of the council_entries
This was 71 councils that we couldn't match to the Shapefile
Completed: 2013_14
We used 0.8655913978494624% of the council_entries
This was 50 councils that we couldn't match to the Shapefile
Completed: 2014_15
We used 0.8655913978494624% of the council_entries
This was 50 councils that we couldn't match to the Shapefile
Completed: 2015_16
We used 0.8387096774193549% of the council_entries
This was 65 councils that we couldn't match to the Shapefile
Completed: 2016_17
We used 0.883289124668435% of the council_entries
This was 44 councils that we couldn't match to the Shapefile
Completed: 2017_18
We used 0.8846153846153846% of the c

/tmp/ipykernel_14799/624293442.py:43: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


Gif saved

Removing Images

DONE


In [ ]:
# frames between transitions
n_frames = 10
x = [1, 2, 3, 4, 5]
coordinates_lists = [[0, 0, 0, 0, 0],
                     [10, 30, 60, 30, 10],
                     [70, 40, 20, 40, 70],
                     [10, 20, 30, 40, 50],
                     [50, 40, 30, 20, 10],
                     [75, 0, 75, 0, 75],
                     [0, 0, 0, 0, 0]]
print('Creating charts\n')
filenames = []
for index in np.arange(0, len(coordinates_lists)-1):
    # get current and next y coordinates
    y = coordinates_lists[index]
    y1 = coordinates_lists[index+1]
    
    # calculate the distance to the next position
    y_path = np.array(y1) - np.array(y)
    for i in np.arange(0, n_frames + 1):
        # divide the distance by the number of frames 
        # and multiply it by the current frame number
        y_temp = (y + (y_path / n_frames) * i)
        # plot
        plt.bar(x, y_temp)
        plt.ylim(0,80)
        # build file name and append to list of file names
        filename = f'images/frame_{index}_{i}.png'
        filenames.append(filename)
        # last frame of each viz stays longer
        if (i == n_frames):
            for i in range(5):
                filenames.append(filename)
        # save img
        plt.savefig(filename)
        plt.close()
print('Charts saved\n')
# Build GIF
print('Creating gif\n')
with imageio.get_writer('mybars.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
print('Gif saved\n')
print('Removing Images\n')
# Remove files
for filename in set(filenames):
    os.remove(filename)
print('DONE')